In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# Set env variables
os.environ["DEPLOY_ENV"] = "voila"
from pathlib import Path

import solara

from sepal_ui.logger import setup_logging
import sepal_ui.sepalwidgets as sw
from sepal_ui.scripts.utils import init_ee
from sepal_ui.sepalwidgets.vue_app import ThemeToggle, MapApp
from sepal_ui.solara.components.admin import AdminButton

from component.widget.custom_widgets import AlertDialog
from component.tile.calculation_tile import CalculationView
from component.tile.dashboard_tile import DashViewA, DashViewB
from component.tile.vegetation_tile import VegetationTile
from component.tile.aoi_tile import AoiView
from component.tile.task_tile import DownloadTaskView
from component.model import MgciModel
from component.message import cm
from component.widget.mgci_map import MgciMap
from component.widget.map import LayerHandler
from sepal_ui.scripts.gee_interface import GEEInterface
from sepal_ui.scripts.drive_interface import GDriveInterface

init_ee()
# Get parent path
logger = setup_logging(logger_name="MGCI", config_path=Path.cwd()/"logging_config.toml")
logger.info("Starting the app")


@solara.component
def Page():

    theme_toggle = ThemeToggle()

    # This is for the secondary panel
    alert = sw.Alert(class_="mt-0 mx-4")
    AlertDialog.element(w_alert=alert)

    gee_interface = GEEInterface()
    drive_interface = GDriveInterface()
    sepal_client = None

    map_ = MgciMap(gee_interface=gee_interface, theme_toggle=theme_toggle)
    aoi_view = AoiView(map_=map_)
    model = MgciModel(aoi_view, sepal_client=sepal_client)
    vegetation_tile = VegetationTile(
        model=model, aoi_model=model.aoi_model, sepal_client=sepal_client, alert=alert
    )
    calculation_tile = CalculationView(
        model=model,
        units="sqkm",
        rsa=True,
        sepal_client=sepal_client,
        gee_interface=gee_interface,
    )
    task_tile = DownloadTaskView(
        sepal_client=sepal_client,
        drive_interface=drive_interface,
        gee_interface=gee_interface,
    )

    layer_handler = LayerHandler(map_, model, alert=alert)
    dash_view_a = DashViewA(model, alert=alert)
    dash_view_b = DashViewB(model, alert=alert)

    steps_data = [
        {
            "id": 2,
            "name": cm.app.drawer_item.aoi,
            "icon": "mdi-map-marker-check",
            "display": "dialog",
            "content": aoi_view,
        },
        {
            "id": 3,
            "name": cm.app.drawer_item.vegetation,
            "icon": "mdi-pine-tree",
            "display": "dialog",
            "content": vegetation_tile,
        },
        {
            "id": 4,
            "name": cm.app.drawer_item.calculation,
            "icon": "mdi-cogs",
            "display": "dialog",
            "content": calculation_tile,
        },
        {
            "id": 5,
            "name": cm.app.drawer_item.dashboard,
            "icon": "mdi-view-dashboard",
            "display": "dialog",
            "content": [],
            "right_panel_action": "toggle",  # "open", "close", "toggle", or None
        },
        {
            "id": 6,
            "name": cm.app.drawer_item.task,
            "icon": "mdi-export",
            "display": "dialog",
            "content": task_tile,
        },
    ]

    right_panel_config = {
        "title": "Results",
        "icon": "mdi-image-filter-hdr",
        "width": 400,
        "description": "This section contains the results of the analysis. You can add layers to the map, view sub-indicators, and download tasks.",
        "toggle_icon": "mdi-chart-line",
    }

    right_panel_content = [
        {
            "title": "Visualize and export layers",
            "icon": "mdi-layers",
            "content": [layer_handler],
            "description": "To add layers to the map, you will first need to select the area of interest and the years in the 3. Indicator settings step.",
        },
        {
            "content": [dash_view_a],
            "title": "Sub indicator A",
            "icon": "mdi-chart-bar",
        },
        {
            "content": [dash_view_b],
            "title": "Sub indicator B",
            "icon": "mdi-chart-bar",
        },
    ]

    MapApp.element(
        app_title="SDG 15.4.2",
        app_icon="mdi-image-filter-hdr",
        main_map=[map_],
        steps_data=steps_data,
        right_panel_config=right_panel_config,
        right_panel_content=right_panel_content,
        theme_toggle=[theme_toggle],
        dialog_width=750,
        repo_url="https://github.com/sepal-contrib/sepal_mgci",
    )

element = Page()
display(element)

<IPython.core.display.Javascript object>

ResizeTrigger()

2025-08-29 14:08:18 - MGCI - INFO - Starting the app
2025-08-29 14:08:18 - sepalui.mapping - DEBUG - Map initialization with gee: True and session: None and interface: <sepal_ui.scripts.gee_interface.GEEInterface object at 0x794abcac8790> ID: 133361899964304
2025-08-29 14:08:18 - sepalui.mapping - DEBUG - Using solara theme: None
2025-08-29 14:08:18 - MGCI.aoi_tile - DEBUG - About to initialize AoiView with map_
2025-08-29 14:08:18 - sepalui - DEBUG - [STARTING <coroutine object GEEInterface.get_folder_async at 0x794ab9ef3ed0>] GEEIterface ID: 133361899964304 || GEE thread: Thread-7 (run_forever) (ID: 133361894057536)
2025-08-29 14:08:18 - sepalui - DEBUG - Running sync coroutine: <coroutine object GEEInterface.get_folder_async at 0x794ab9ef3ed0>
2025-08-29 14:08:18 - sepalui - DEBUG - Sync coroutine completed successfully: <coroutine object GEEInterface.get_folder_async at 0x794ab9ef3ed0>
2025-08-29 14:08:18 - sepalui.sepalwidgets.inputs - DEBUG - INITIALIZING AssetSelect 133361457812

Cannot show ipywidgets in text

2025-08-29 14:08:19 - sepalui.sepalwidgets.inputs - DEBUG - After validating the v_model is users/amitghosh/sdg_module/esa/cci_landcover for AssetSelect
2025-08-29 14:08:19 - sepalui.sepalwidgets.inputs - DEBUG - After validating the v_model is users/amitghosh/sdg_module/esa/cci_landcover for AssetSelect
2025-08-29 14:08:19 - MGCI.reclassify_view - DEBUG - QQQQQQQQQQQQQQQQQQQQQQ Setting ids
2025-08-29 14:08:19 - MGCI.model - DEBUG - MgciModel: ic_items_sub_a changed from [] to ['users/amitghosh/sdg_module/esa/cci_landcover/1992', 'users/amitghosh/sdg_module/esa/cci_landcover/1993', 'users/amitghosh/sdg_module/esa/cci_landcover/1994', 'users/amitghosh/sdg_module/esa/cci_landcover/1995', 'users/amitghosh/sdg_module/esa/cci_landcover/1996', 'users/amitghosh/sdg_module/esa/cci_landcover/1997', 'users/amitghosh/sdg_module/esa/cci_landcover/1998', 'users/amitghosh/sdg_module/esa/cci_landcover/1999', 'users/amitghosh/sdg_module/esa/cci_landcover/2000', 'users/amitghosh/sdg_module/esa/cci_land